In [8]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [9]:
from tensorflow import keras
from tensorflow.keras import layers, models, Input

In [10]:
#層的權重共享
#函數式API其中一個重要特性是能重複使用層物件
#有個評估兩個句子語義相似與否的模型，此模型有兩個輸入(兩個要比較的句子)，輸出0~1之間的分數，其中0表示不相關，1則表示完全相同或僅是重新排列的句子
#此模型在對話系統中，可去除重複的語句
#此案例中兩個輸入句子是可互換的，因為語義相似性是對稱關係:A到B的相似性與B到A的相似性相同，因此我們可以使用單個LSTM層處理這兩個句子
#這個LSTM層的表示法(權重)是基於兩個輸入同時學習，也就是孿生LSTM(Siamese LSTM)或共享LSTM(shared LSTM)

In [12]:
lstm = layers.LSTM(32) #建立一個神經元32個的LSTM物件

left_input = Input(shape=(None, 128)) #建立模型的左輸入分支:可變長度的向量，大小為128
print(left_input.shape) #左分支的輸入張量shape=(?,?,128)
left_output = lstm(left_input) #傳入LSTM物件
print(left_output.shape) #左分支的輸出張量shape=(?,32)

right_input = Input(shape=(None, 128)) #建立模型的右輸入分支:可變長度的向量，大小為128
print(right_input.shape) #右分支的輸入張量shape=(?,?,128)
right_output = lstm(right_input) #傳入"同一個"LSTM物件
print(right_output.shape) #右分支的輸出張量shape=(?,32)

(None, None, 128)
(None, 32)
(None, None, 128)
(None, 32)


In [13]:
#將向量串接
merged = layers.concatenate([left_output, right_output], axis=-1)
print(merged.shape) #串接後向量shape=(?,32+32=64)

(None, 64)


In [14]:
#將串接後的向量傳入最後的輸出層Dense進行分類
predictions = layers.Dense(1, activation='sigmoid')(merged)

In [16]:
#用輸入向量(2個)與輸出向量建立Model物件
model = models.Model([left_input, right_input], predictions)

In [17]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, None, 128)]  0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, None, 128)]  0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   (None, 32)           20608       input_3[0][0]                    
                                                                 input_4[0][0]                    
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 64)           0           lstm_1[0][0]               

In [ ]:
#訓練模型
model.fit([left_data, right_data], labels)
#當訓練這樣的模型時，LSTM層的權重將同時依據兩個輸入進行更新

In [19]:
#將模型作為層
#函數式API中最為重要的功能是，能將模型A放到另個模型B中來做使用，Sequential和Model類別都能使用
#用輸入張量(x)直接呼叫模型物件(視為一個層)，並取得輸出張量(y)
#y = model(x)
#y1, y2 = model([x1, x2])
#呼叫一個物件時，無論是"層物件"或是"模型物件"，將會重複使用物件現已學習到的表示法(權重)

In [20]:
#一個使用雙攝影鏡頭影像作為輸入的視覺模型，即設置兩個並行(相距幾公分)的攝影鏡頭
#此模型可以感知影像深度，延伸許多應用，ex:3D人臉辨識
#使用同一個層對左、右鏡頭一起進行萃取其視覺特徵，從而共享相同的表示法(權重)，稱為孿生(Siamese)視覺模型(共享卷積基礎)
from tensorflow.keras.applications import Xception

xception_base = Xception(weights=None, include_top=False) #使用Xception神經網路的卷積基底(不包含最上層分類器)進行影像特徵萃取

#建立左、右輸入張量(左、右鏡頭影像)，其shape=(250,250,3)，即為250*250的彩色影像
left_input = Input(shape=(250, 250, 3))
right_input = Input(shape=(250, 250, 3))

#呼叫相同的視覺模型兩次，將左、右影像張量傳入Xception神經網路物件
left_features = xception_base(left_input)
right_features = xception_base(right_input)

print(left_features.shape)
print(right_features.shape)

(None, 8, 8, 2048)
(None, 8, 8, 2048)


In [22]:
#串接左右影像特徵張量
merged_features = layers.concatenate([left_features, right_features], axis=-1)
print(merged_features.shape)

(None, 8, 8, 4096)
